In [13]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [14]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale if not already
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

train_data_path = 'double_mnist/train'
val_data_path = 'double_mnist/val'
test_data_path = 'double_mnist/test'

# Create ImageFolder datasets for training, validation, and test sets
train_dataset = ImageFolder(train_data_path, transform=transform)
val_dataset = ImageFolder(val_data_path, transform=transform)
test_dataset = ImageFolder(test_data_path, transform=transform)

class_to_label = {str(i).zfill(2): i for i in range(100)}
# Define batch sizes
batch_size = 64

# Create DataLoader instances for training, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Number of classes
num_classes = 10

class_names = train_dataset.classes
class_names = val_dataset.classes

print(class_names)






['03', '07', '10', '22', '27', '34', '39', '40', '48', '52', '58', '61', '64', '71', '93', '99']


In [24]:
class SeparatedDigitCNN(nn.Module):
    def __init__(self, kernel_size=3, pool_size=2, stride=1, dropout_rate=0.5):
        super(SeparatedDigitCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=int((kernel_size - 1) / 2))
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=pool_size, stride=stride)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=int((kernel_size - 1) / 2))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=pool_size, stride=stride)
        var1 = int((28 - pool_size) / stride) + 1
        self.dim = int((var1 - pool_size) / stride) + 1
        self.fc1_digit1 = nn.Linear(64 * self.dim * self.dim, num_classes)
        self.fc1_digit2 = nn.Linear(64 * self.dim * self.dim, num_classes)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * self.dim * self.dim)
        x1 = self.fc1_digit1(x)
        x2 = self.fc1_digit2(x)
        return x1, x2
    
model = SeparatedDigitCNN()

# Define loss functions for each digit separately
criterion_digit1 = nn.CrossEntropyLoss()
criterion_digit2 = nn.CrossEntropyLoss()

# Define a single optimizer for both digit branches
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [29]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_train_loss_digit1 = 0
    total_train_loss_digit2 = 0
    total_hamming_loss = 0
    total_correct_predictions = 0
    total_predictions = 0
    cnt = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs_digit1, outputs_digit2 = model(images)

        # Extract the individual digits from the labels
        labels_digit1 = labels // 10
        labels_digit2 = labels % 10

        loss_digit1 = criterion_digit1(outputs_digit1, labels_digit1)
        loss_digit2 = criterion_digit2(outputs_digit2, labels_digit2)
        total_train_loss_digit1 += loss_digit1.item()
        total_train_loss_digit2 += loss_digit2.item()

        # Combine predicted results into a list of 20 elements
        predicted_list = torch.cat((outputs_digit1, outputs_digit2), dim=1)
        # print(outputs_digit1)
        # predicted_labels_digit1 = torch.argmax(outputs_digit1, dim=1)
        # predicted_labels_digit2 = torch.argmax(outputs_digit2, dim=1)
        # print(predicted_list)
        # Calculate Hamming loss
        # predicted_labels = torch.argmax(predicted_list, dim=1)
        predicted_labels_digit1 = torch.argmax(predicted_list[:, :10], dim=1)
        predicted_labels_digit2 = torch.argmax(predicted_list[:, 10:], dim=1)
        # hamming_loss = (predicted_labels != labels).sum().item() / (labels.size(0) * 20)
        hamming_loss_digit1 = (predicted_labels_digit1 != labels_digit1).float().mean()
        hamming_loss_digit2 = (predicted_labels_digit2 != labels_digit2).float().mean()
        hamming_loss = (hamming_loss_digit1+hamming_loss_digit2)/2
        if(hamming_loss < 1):
            cnt += 1
            print(predicted_labels.shape)
            print(labels.shape)
            # print(predicted_labels)
            print(predicted_labels)
            print(labels)
            if(cnt == 5):
                break
        total_hamming_loss += hamming_loss

        # Calculate overall accuracy
        total_correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0) * 20

    train_accuracy = 100 * total_correct_predictions / total_predictions
    average_train_loss_digit1 = total_train_loss_digit1 / len(train_loader)
    average_train_loss_digit2 = total_train_loss_digit2 / len(train_loader)
    average_hamming_loss = total_hamming_loss / len(train_loader)

    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'Digit 1 - Train Loss: {average_train_loss_digit1:.4f}')
    print(f'Digit 2 - Train Loss: {average_train_loss_digit2:.4f}')
    print(f'Hamming Loss: {average_hamming_loss:.4f}')
    print(f'Train Accuracy: {train_accuracy:.2f}%')

torch.Size([64])
torch.Size([64])
tensor([ 1,  1, 15,  0,  0, 12,  0, 18, 12, 12, 11, 11, 15, 11, 12, 15, 11, 12,
        11, 11, 12, 18, 11, 15, 15, 18, 11,  0, 11, 11, 11, 11, 12, 15, 18, 16,
        16, 16, 16, 18, 16, 12, 11, 11, 16, 16, 11, 11, 11, 12,  0, 15, 11,  1,
         0, 11, 15, 15,  0, 15, 16, 18,  2,  0])
tensor([57, 22, 17, 14, 42, 35, 61, 28, 39, 27, 39, 40, 44, 26, 47, 62, 61, 36,
        35, 39, 35, 50,  3, 25, 40, 38, 23, 43,  3, 45, 60, 34, 59, 54, 20, 29,
        28, 41, 26, 18, 58,  8, 51,  3, 22, 27, 44,  7, 46, 60, 61, 50, 11, 32,
        59, 51, 58, 12,  3, 16, 46, 47, 28, 53])
torch.Size([64])
torch.Size([64])
tensor([ 1,  1, 15,  0,  0, 12,  0, 18, 12, 12, 11, 11, 15, 11, 12, 15, 11, 12,
        11, 11, 12, 18, 11, 15, 15, 18, 11,  0, 11, 11, 11, 11, 12, 15, 18, 16,
        16, 16, 16, 18, 16, 12, 11, 11, 16, 16, 11, 11, 11, 12,  0, 15, 11,  1,
         0, 11, 15, 15,  0, 15, 16, 18,  2,  0])
tensor([51, 61,  5, 41, 30,  2, 10, 21, 34, 24, 58, 44, 59, 58, 3